In [1]:
# %pip install --upgrade xlrd
# %pip install pyarrow
# %pip install openpyxl
# %pip install yahoo_fin
# %pip install alpha_vantage
# %pip install yfinance

import pandas as pd
import numpy as np
import requests
import time
from datetime import datetime, date, timedelta
from dateutil.parser import parse
from dateutil.relativedelta import relativedelta
from functools import reduce

# import yfinance as yf
# from yahoo_fin.stock_info import get_data
# from alpha_vantage.timeseries import TimeSeries

In [2]:
#pd.set_option("display.max_rows", None, "display.max_columns", None)
now = datetime.now()
american_time = datetime.strftime(now, '%Y-%m-%d')

In [18]:
# Federal Reserve Data - https://fred.stlouisfed.org/

FED_series_list = ['DFF', 'MORTGAGE30US', 'DPRIME', 'DTB3', 'DTB1YR', 
                    'DTB6', 'DGS30', 'DGS20', 'DGS10', 'DGS7', 
                    'DGS5', 'DGS3', 'DGS2', 'DGS1', 'DGS3MO', 'DGS6MO', 
                    'CPIAUCSL', 'M2SL', 'A191RP1Q027SBEA', 'UNRATE', 
                    'A191RL1Q225SBEA', 'MSPUS', 'M1SL', 'PSAVERT', 
                    'CIVPART', 'WAAA', 'PCE', 'NFCI', 'MTSDS133FMS',
                    'GFDEGDQ188S', 'MEHOINUSA672N', 'ICSA', 'CURRCIR', 
                    'BOGMBASE', 'INDPRO', 'RECPROUSM156N']

joining_dict = {series : pd.read_excel(f'https://fred.stlouisfed.org/graph/fredgraph.xls?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1168&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id={series}&scale=left&cosd=1984-01-07&coed={american_time}&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Daily&fam=avg&fgst=lin&fgsnd={american_time}&line_index=1&transformation=lin&vintage_date={american_time}&revision_date={american_time}&nd=1984-01-07',
                    header=10, 
                    names=['date', series]) 
                    for series in FED_series_list}
start_date = max([df['date'].min() for df in joining_dict.values()])
joining_dict = {key : df[df['date'] >= start_date] for key, df in zip(joining_dict.keys(), joining_dict.values())}

FED_data = reduce(lambda left,right: pd.merge_asof(left, right, on='date'), joining_dict.values(), joining_dict.pop('DFF'))
FED_data = FED_data.fillna(method='ffill')
FED_data = FED_data[FED_data.isna().any(axis=1) == False].reset_index(drop=True).copy()
FED_data.to_feather('Data/FED_data.ftr')

In [4]:
# # Historical SPX data stored in Excel file as backup

# df = pd.read_excel('Data/SPX.xlsx')
# df['Date'] = df['Date'].apply(parse)
# df.columns = [col.lower().replace('*','') for col in df.columns]
# df['high-low'] = df['high']-df['low']
# df = df[['date', 'close', 'high-low']].copy()

# df = df.sort_values(by="date")
# df = df.reset_index(drop=True)
# df.to_feather('Data/SPX.ftr')

In [11]:
# https://www.marketwatch.com/investing/index/spx/download-data

df_old = pd.read_feather('Data/SPX.ftr')
last_date = df_old['date'].max() + relativedelta(days=1)

df = pd.read_csv('https://www.marketwatch.com/investing/index/spx/downloaddatapartial?startdate={}%2000:00:00&enddate={}%2023:59:59&frequency=p1d&csvdownload=true'
                        .format(last_date.strftime('%m/%d/%Y'), 
                                now.strftime('%m/%d/%Y')))

df.columns = [col.lower() for col in df.columns]
for col in [i for i in df.columns if i != 'date']:
        df[col] = pd.to_numeric(df[col].apply(lambda x: x.replace(',',''))).copy()

df['high-low'] = df['high'] - df['low']
df['date'] = df['date'].apply(lambda x: datetime.strptime(x, '%m/%d/%Y'))
df = df.drop(['open','high','low'], axis=1)

df.columns = df_old.columns
df = df.sort_values(by='date')
df = df.reset_index(drop=True)

df = pd.concat((df_old, df), ignore_index=True)
df.to_feather('Data/SPX.ftr')

In [6]:
# https://download.bls.gov/pub/time.series/cu/cu.txt

CPI_types = ['11.USFoodBeverage', '12.USHousing', '13.USApparel', '14.USTransportation', '15.USMedical', '16.USRecreation', '17.USEducationAndCommunication', '18.USOtherGoodsAndServices', '20.USCommoditiesServicesSpecial']
CPI_dict = {}

for name in CPI_types:
    df = pd.read_csv(f'https://download.bls.gov/pub/time.series/cu/cu.data.{name}', delimiter='\t')
    df = df.iloc[:,:4]
    df.columns = ['series_id','year','month','CPI']
    df['month'] = df['month'].apply(lambda x: int(x[1:]))
    df = df[df['month'] != 13]
    df = df.reset_index(drop=True)

    # Calculating the mean CPI for each of the categories in CPI_types,
    # and filling a dictionary with those series.
    new_CPI = []
    for y in df['year'].unique():
        for m in df['month'].unique():
            year = df[df['year']==y]
            month = year[year['month']==m]
            thingy = (y, m, month['CPI'].mean(skipna=True))
            new_CPI.append(thingy)
    new_name = name.split('.')[1]
    CPI_dict[f'{name}'] = pd.DataFrame(new_CPI, columns=['year', 'month', f'CPI_{new_name}'])


df = CPI_dict[list(CPI_dict.keys())[0]][['year', 'month']]
for key in list(CPI_dict.keys()):
    df[key] = CPI_dict[key].iloc[:,2]

df['day'] = 1
date_creation = df[['year', 'month', 'day']].astype(str).copy()
date_creation['date'] = date_creation['year']+'-'+date_creation['month']+'-'+date_creation['day']
df['date'] = date_creation['date'].apply(parse)
df = df.drop(['year','month','day'], axis=1)

df = df.sort_values(by='date')
df = df.reset_index(drop=True)
df.to_feather('Data/CPI.ftr')

In [22]:
# Merging all datasets into 1 dataframe
file_list = [file.replace('.ftr','') for file in os.listdir('Data') if '.ftr' in file and file != 'cleaned_dataset.ftr']
joining_dict = {filename : pd.read_feather(f'Data/{filename}.ftr') for filename in file_list}
start_date = max([df['date'].min() for df in joining_dict.values()])
joining_dict = {key : df[df['date'] >= start_date] for key, df in zip(joining_dict.keys(), joining_dict.values())}

joined = reduce(lambda left,right: pd.merge_asof(left, right, on='date'), joining_dict.values(), joining_dict.pop('SPX'))
joined = joined[joined['date'] <= now].reset_index(drop=True).rename(columns={'close':'SPX_close'})

In [23]:
# Feature engineering
joined = pd.merge_asof(joined, pd.DataFrame({"date":joined.date, "SPX_close-7":joined['SPX_close'].rolling(window=7).mean()}).reset_index(drop=True), on='date')
joined = pd.merge_asof(joined, pd.DataFrame({"date":joined.date, "SPX_close-15":joined['SPX_close'].rolling(window=15).mean()}).reset_index(drop=True), on='date')

joined = joined.fillna(method='ffill')
joined = joined.fillna(method='bfill')

DFF_delta = [] # Creating a column to signal when (and how) the Fed changes rate
percent_change = 0.03
SPX_drop = [] # Creating a column to signal when, in the course of the next week, the SPX drops by more than percent_change% from current level
SPX_bump = [] # Creating a column to signal when, in the course of the next week, the SPX increases by more than percent_change% from current level
for i in range(len(joined) - 1):
    DFF_delta.append(joined['DFF'][i+1] - joined['DFF'][i])   
    drop = False
    bump = False
    for j in range(7):
        if i+j+1 < len(joined):
            if not drop:
                drop = (joined['SPX_close-7'][i] - joined['SPX_close-7'][i+j+1]) > percent_change*joined['SPX_close-7'][i]
            if not bump:
                bump = (joined['SPX_close-7'][i+j+1] - joined['SPX_close-7'][i]) > percent_change*joined['SPX_close-7'][i]
    SPX_drop.append(int(drop))
    SPX_bump.append(int(bump))
DFF_delta.append(0)
SPX_drop.append(0)
SPX_bump.append(0)
joined['DFF_delta'] = DFF_delta
joined['SPX_drop'] = SPX_drop
joined['SPX_bump'] = SPX_bump

joined.to_feather('Data/cleaned_dataset.ftr')

In [24]:
joined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9455 entries, 0 to 9454
Data columns (total 53 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   date                             9455 non-null   datetime64[ns]
 1   SPX_close                        9455 non-null   float64       
 2   high-low                         9455 non-null   float64       
 3   11.USFoodBeverage                9455 non-null   float64       
 4   12.USHousing                     9455 non-null   float64       
 5   13.USApparel                     9455 non-null   float64       
 6   14.USTransportation              9455 non-null   float64       
 7   15.USMedical                     9455 non-null   float64       
 8   16.USRecreation                  9455 non-null   float64       
 9   17.USEducationAndCommunication   9455 non-null   float64       
 10  18.USOtherGoodsAndServices       9455 non-null   float64    